In [77]:
#########################################
# Toy implementation of factoring algorithm (Algorithm 2.2) in 
# "Factoring using multiplicative relations modulo n:
#      a subexponential factoring algorithm inspired by the index calculus
# by Katherine E. Stange
#########################################

def get_order(n, g, B=10, rels=10, verbose=True, alphas=False, texify=False):
    # INPUT:
    # n = modulus
    # g = base
    # B = bound for primes
    # rels = number of additional relations
    # verbose = whether to print info as you go
    # alphas = whether to return the alpha values
    # textify = whether to print latex for algorithm
    # OUTPUT:  order of g modulo n (with high probability)
    
    # Set up factor base
    if verbose:
        print("Attempting to compute the order of ", g, "modulo", n)
    numPrimes = prime_pi(B)  # the number of primes <= B
    if verbose:
        print("The number of primes in the factor base:", numPrimes)
        print("These are:", [nth_prime(m) for m in range(1, numPrimes+1)])
    
    # Set the number of relations to find
    relsDesired = numPrimes + rels  # the number of relations to find
    if verbose:
        print("We are looking for ", relsDesired, "relations.")
    vectors = []  # to store the relations we find
        
    # Main relation finding loop
    searchCount = 0 # number of smoothness tests run
    relsFound = 0 # number of relations found
    while( relsFound < relsDesired ):
        
        # take a random power of x
        x = randint(1,n)
        residue = ZZ(Mod(g,n)^x)
        
        # trial division to test smoothness
        fac = [0 for _ in range(B+1)]
        for p in primes(B):
            while Mod(residue,p) == 0:
                residue = residue/p
                fac[p] += 1
        
        if residue == 1:  # store a relation if smooth
            if verbose:
                print("found a relation:", g, "^", x, "is", factor(ZZ(Mod(g,n)^x)))
            if texify:
                print(g,"^{",x,"}&=",latex(factor(ZZ(Mod(g,n)^x))),",\\\\")
            vec = vector([fac[nth_prime(m)] for m in range(1,numPrimes+1)])
            vectors.append([vec,x])
            relsFound += 1
            
        searchCount += 1
        
    if verbose:
        print("I searched through ", searchCount, "powers of ", g)
        
    # Linear algebra phase
    relMatrix = matrix([vectors[i][0] for i in range(relsDesired)]).transpose()
    if verbose:
        print("The relation matrix is (cols are relations):")
        print(relMatrix)
    if texify:
        print(latex(relMatrix))
    kernel = relMatrix.right_kernel().basis()
    if verbose:
        print("The right kernel has basis:")
        print(kernel)
    if texify:
        print(latex(matrix(kernel)))
    alphaVals = []
    for basisVec in kernel:
        alpha = sum([basisVec[i]*vectors[i][1] for i in range(relsDesired)])
        alphaVals.append(alpha)
    if verbose:
        print("The corresponding sums of the x's are:")
        print(alphaVals)
    if texify:
        print(latex(alphaVals))
        
    # GCD phase
    finalGCD = 0
    for alpha in alphaVals:
        finalGCD = gcd(finalGCD,alpha)
        
    # Report and return
    if verbose:
        print("Their gcd is:", finalGCD)
        print("******* Relation to reality? ******")
        print("Sage's factorization of", n, "is", factor(n))
        print("The actual multiplicative order is:", Mod(g,n).multiplicative_order())
        if Mod(g,n).multiplicative_order() == finalGCD:
            print("Success!")
        else:
            print("Failure!")
            print("The ratio of the expected value to the real value is (1=success):", finalGCD/Mod(g,n).multiplicative_order())
    if alphas:
        return(finalGCD,[a/finalGCD for a in alphaVals])
    return(finalGCD)
    
            
    

In [76]:
get_order(701*89, g=43, B=50, verbose=True, texify=True)

Attempting to compute the order of  43 modulo 62389
The number of primes in the factor base: 15
These are: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
We are looking for  25 relations.
found a relation: 43 ^ 51838 is 2 * 3^3 * 7^3
43 ^{ 51838 }&= 2 \cdot 3^{3} \cdot 7^{3} ,\\
found a relation: 43 ^ 35603 is 2^2 * 7^2 * 11 * 23
43 ^{ 35603 }&= 2^{2} \cdot 7^{2} \cdot 11 \cdot 23 ,\\
found a relation: 43 ^ 40767 is 3^3 * 5^2 * 11
43 ^{ 40767 }&= 3^{3} \cdot 5^{2} \cdot 11 ,\\
found a relation: 43 ^ 30297 is 2^2 * 5 * 7^2 * 41
43 ^{ 30297 }&= 2^{2} \cdot 5 \cdot 7^{2} \cdot 41 ,\\
found a relation: 43 ^ 54687 is 2 * 3 * 43^2
43 ^{ 54687 }&= 2 \cdot 3 \cdot 43^{2} ,\\
found a relation: 43 ^ 61970 is 2^2 * 11 * 47
43 ^{ 61970 }&= 2^{2} \cdot 11 \cdot 47 ,\\
found a relation: 43 ^ 1685 is 3^2 * 11^2 * 31
43 ^{ 1685 }&= 3^{2} \cdot 11^{2} \cdot 31 ,\\
found a relation: 43 ^ 24370 is 2^4 * 3^2 * 7 * 31
43 ^{ 24370 }&= 2^{4} \cdot 3^{2} \cdot 7 \cdot 31 ,\\
found a relation: 43 ^ 6

15400